# Conditions for local minima

The necessary conditions for $\mathbf{k^*}$ being a local minimizer is
\begin{equation}
\mathbf{g}(\mathbf{k^*})=\mathbf{0}
\end{equation}
and
\begin{equation}
\mathbf{s}^\top\mathbf{H}(\mathbf{k^*})\mathbf{s}\geq 0,\hspace{5mm}\forall\mathbf{s}
\end{equation}

The sufficient condition for a strict and isolated local minimizer $\mathbf{k^*}$ is that $\mathbf{H}(\mathbf{k^*})$ is positive definite,
\begin{equation}
\mathbf{s}^\top\mathbf{H}(\mathbf{k^*})\mathbf{s}>0\hspace{5mm}\forall\mathbf{s}\neq\mathbf{0}
\end{equation}

For the numerical check of positive definite matrix, Cholesky factorization is usefull. Matrix $\mathbf{H}$ is positive definite when the Cholesky factor
\begin{equation}
\mathbf{H}=\mathbf{L}\mathbf{L}^\top
\end{equation}
exits and all $l_{ii}>0$.

In [ ]:
def cholesky(A, p):
    C = np.zeros((p,p))
    j = 0
    pd = True
    
    while pd and j < p:
        sum = 0
        
        for k in range(j):
            sum += C[j, k]**2
            
        d = A[j, j] - sum
        
        if d > 0:
            C[j, j] = np.sqrt(d)
            
            for i in range(j,p):
                sum = 0
                for k in range(j):
                    sum += C[i, k] * C[j, k]
                C[i, j] = (A[i, j]-sum) / C[j, j]
                
        else:
            pd = False
            
        j += 1
        
    return C, pd



def cholsolve(A, b, mu, p):
    I = np.eye(p)
    mA = np.amax(abs(A))
    
    
    if mu != 0:
        pd = False

        while pd == False:
            C, pd = cholesky(A + mu * I, p)

            # check for near singularity
            if pd == True:
                pd = (1 / LA.cond(C,1) >= 1e-15)
            if pd == False:
                mu = max(10 * mu, np.finfo(float).eps * mA)
    else:
        C, pd = cholesky(A, p)
        assert pd, "non positive definite A"
            
            
    # CC^Tx = b
    z = np.zeros(p)
    x = np.zeros(p)
    # Forward C^Tx = z
    
    for i in range(p):
        sum = 0
        for j in range(i):
            sum += C[i][j] * z[j]
            
        z[i] = (b[i]-sum) / C[i][i]
        
    # Backward Cz = b
    for i in reversed(range(p)):
        sum = 0
        for j in range(i,p):
            sum += C[j][i] * x[j]
            
        x[i] = (z[i]-sum) / C[i][i]
        
    return x, mu